# Industry energy demand

### Import necessary modules

In [1]:
# Check if we are running the notebook directly, if so move workspace to parent dir
import sys
import os
currentdir = os.path.abspath(os.getcwd())
if os.path.basename(currentdir) != 'DemandMappingZambia':  
  sys.path.insert(0, os.path.dirname(currentdir))
  os.chdir('..')
  print(f'Move to {os.getcwd()}')

Move to C:\Users\amillot\PycharmProjects\DemandMappingZambia


In [2]:
### Activate geospatial_env first

# Numeric
import numpy as np
import pandas as pd
import math

# System
import os
import shutil
from IPython.display import display, Markdown, HTML, FileLink, FileLinks

# Spatial
import geopandas as gpd
import json
import pyproj
from shapely.geometry import Point, Polygon, MultiPoint
from shapely.wkt import dumps, loads
from shapely.ops import nearest_points
from pyproj import CRS
import ogr, gdal, osr
#import fiona


# Mapping / Plotting
from functools import reduce
#import datapane as dp 
#!datapane login --token="9bde41bfbc4ad14119e32086f9f06d2e5db1d5b8"
import folium
from folium.features import GeoJsonTooltip
from folium.plugins import BeautifyIcon
from folium.plugins import HeatMap
import branca.colormap as cm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

%matplotlib inline

In [3]:
#import geopandas as gpd   # Note that you require geopandas version >= 0.7 that incluse clip see here for installation (https://gis.stackexchange.com/questions/360127/geopandas-0-6-1-installed-instead-of-0-7-0-in-conda-windows-10#)
import os
import fiona
import ipywidgets as widgets
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
from geojson import Feature, Point, FeatureCollection
import rasterio.fill
from shapely.geometry import shape, mapping
import json
#from earthpy import clip    clip has been deprecated to geopandas
#import earthpy.spatial as es
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import datetime
import warnings
import pandas as pd
import scipy.spatial
warnings.filterwarnings('ignore')

#import contextily as ctx
import h3 as h3
from shapely.ops import unary_union
from shapely.geometry.polygon import Polygon

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

pd.options.display.float_format = '{:,.4f}'.format

In [4]:
from utils import processing_raster, finalizing_rasters, spatialjoinvectors

### Define directories and dataset names

In [5]:
### Define directories and dataset names
ROOT_DIR = os.path.abspath(os.curdir)
in_path = ROOT_DIR
out_path = ROOT_DIR + "/Outputs/"

## mines layer
ind_data_path = in_path + "/Industry/Data/"
mines_path = ind_data_path + "mines"
# mines_name = 'mines_zambia.gpkg'
mines_name = 'mineral_facilities_zambia_May_2024'
file_mines = 'Mineral_Facilities_correctedInput.csv'

In [6]:
## Coordinate and projection systems
crs_WGS84 = CRS("EPSG:4326")    # Originan WGS84 coordinate system
crs_proj = CRS("EPSG:32736")    # Projection system for the selected country -- see http://epsg.io/ for more info

In [7]:
ROOT_DIR

'C:\\Users\\amillot\\PycharmProjects\\DemandMappingZambia'

### Define area of interest

In [8]:
try: area
except NameError: area = "COUNTRY"
# area = "Copperbelt"

### Import layers to be used

In [9]:
hexagons = gpd.read_file(out_path + "h3_grid_at_hex_7.shp")

In [10]:
grid = hexagons

In [11]:
## mines
# mines = gpd.read_file(mines_path + mines_name)

In [12]:
# ## admininstrative boundary
admin_gdf = gpd.read_file(ROOT_DIR + "/admin/" + "gadm41_ZMB.gpkg")


In [16]:
admin_gdf_region = gpd.read_file(ROOT_DIR + "/admin/gadm41_ZMB_shp/" + "gadm41_ZMB_1.shp")

In [17]:
admin_gdf_region

,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry
0,ZMB.1_1,ZMB,Zambia,Central,Broken Hill,NA,Province,Province,NA,ZM.CE,ZM-02,"POLYGON ((27.24544 -15.72238, 27.24094 -15.723..."
1,ZMB.2_1,ZMB,Zambia,Copperbelt,Western,NA,Province,Province,NA,ZM.CO,ZM-08,"POLYGON ((28.64984 -12.83038, 28.65274 -12.833..."
2,ZMB.3_1,ZMB,Zambia,Eastern,Fort Jameson,NA,Province,Province,NA,ZM.EA,ZM-03,"POLYGON ((30.78534 -14.78948, 30.77554 -14.793..."
3,ZMB.4_1,ZMB,Zambia,Luapula,Fort Rosebery,NA,Province,Province,NA,ZM.LP,ZM-04,"POLYGON ((29.57224 -12.19608, 29.56986 -12.196..."
4,ZMB.5_1,ZMB,Zambia,Lusaka,NA,NA,Province,Province,NA,ZM.LS,ZM-09,"POLYGON ((28.90991 -15.98370, 28.90808 -15.987..."
5,ZMB.6_1,ZMB,Zambia,Muchinga,NA,NA,Province,Province,NA,ZM.MU,ZM-10,"POLYGON ((29.81347 -12.14948, 29.81345 -12.149..."
6,ZMB.7_1,ZMB,Zambia,North-Western,NA,NA,Province,Province,NA,ZM.NW,ZM-06,"POLYGON ((23.53308 -12.99530, 23.94690 -12.997..."
7,ZMB.8_1,ZMB,Zambia,Northern,NA,NA,Province,Province,NA,ZM.NO,ZM-05,"POLYGON ((29.81334 -11.41218, 29.90984 -11.011..."
8,ZMB.9_1,ZMB,Zambia,Southern,NA,NA,Province,Province,NA,ZM.SO,ZM-07,"POLYGON ((25.89364 -17.97668, 25.88814 -17.975..."
9,ZMB.10_1,ZMB,Zambia,Western,Barotseland,NA,Province,Province,NA,ZM.WE,ZM-01,"POLYGON ((24.95344 -17.54978, 24.94784 -17.551..."


In [14]:
# transmission lines
grid_gdf = gpd.read_file(ROOT_DIR + "/Grid/Zambia - HVLines/" + "HVLines.shp")

In [15]:
grid_gdf

,OBJECTID,ENABLED,CREATIONUS,DATECREATE,DATEMODIFI,LASTUSER,SUBTYPECD,MEASUREDLE,LENGTHSOUR,WORKORDERI,...,circuits,line,designatio,power_line,material,structure,fixme,kv,Voltage_1,geometry
0,1281,1,NaN,NaN,NaN,NaN,3,25.0000,NaN,NaN,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.87926 -14.09954, 31.87926 -14.0..."
1,1282,1,NaN,NaN,NaN,NaN,3,0.0000,NaN,NaN,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.87928 -14.09958, 31.87928 -14.0..."
2,1283,1,NaN,NaN,NaN,NaN,3,"2,958.0000",NaN,NaN,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.87318 -14.07787, 31.87383 -14.0..."
3,1284,1,NaN,NaN,NaN,NaN,3,"9,053.0000",NaN,NaN,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.88261 -14.05285, 31.88320 -14.0..."
4,1285,1,NaN,NaN,NaN,NaN,3,"2,006.0000",NaN,NaN,...,10,10,10,10,10,10,10,0.0000,66.0000,"LINESTRING (31.90315 -13.97378, 31.90356 -13.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,4,2,10,NaN,NaN,10,4,6.0000,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (25.86684 -12.25216, 25.86747 -12.2..."
927,4,2,10,NaN,NaN,10,4,6.0000,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (25.86694 -12.25200, 25.86753 -12.2..."
928,4,2,10,NaN,NaN,10,4,6.0000,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (28.75631 -16.51335, 28.75682 -16.5..."
929,4,2,10,NaN,NaN,10,4,6.0000,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0000,92.0000,"LINESTRING (24.67206 -17.27949, 24.67161 -17.2..."


# Part 1. Extract GIS-based attributes

## Extract information from vector layers

Assess the energy consumption per site

In [13]:
## Attribute energy consumption to each copper production site
mines_input_file = pd.read_csv(ind_data_path + file_mines)

In [14]:
## Running the energy calculation function 
from energy_per_site import *
calc_energy_per_site(mines_path, mines_input_file, mines_name)

Production at a site  Fitwaola Open Pit Mine  in  Zambia  is missing (negative). Value set to zero. Please, change the input in the input file.
Production at a site  Mkushi Heap Leach  in  Zambia  is missing (negative). Value set to zero. Please, change the input in the input file.
Production at a site  Mufulira West Heap-Leach Facility  in  Zambia  is missing (negative). Value set to zero. Please, change the input in the input file.


Extract sum production of mines in each cluster (hex)

In [15]:
## Run the extraction
#grid.drop(['Commodity Production - tonne (tonnes)'], axis=1, inplace=True) ##uncomment if you want to rerun
# columnNameMines = "Commodity Production - tonne (tonnes)"
#columnNameMines = 'Ore processed (tonnes)'
# grid[columnNameMines] = grid[columnNameMines].fillna(0)
# grid.head(4)
# grid[columnNameMines].sum()

columnName_ElecConsumptionMines = 'Energy Elec [TJ]'
file = os.path.join(mines_path, mines_name +".gpkg")
grid, mines = spatialjoinvectors("EnergyMines", columnName_ElecConsumptionMines, admin_gdf, crs_WGS84, grid, "sum", file)

In [16]:
grid[columnName_ElecConsumptionMines].sum()

17986.613885878785

In [17]:
grid.head(4)

,hexagons,lon,lat,index_righ,NAME_1,id,geometry,Energy Elec [TJ]
0,87399485cffffff,30.5240,-12.1759,5.0000,Muchinga,1,"POLYGON ((30.53691 -12.18031, 30.52690 -12.190...",NaN
1,873995215ffffff,30.4244,-10.4257,7.0000,Northern,2,"POLYGON ((30.43731 -10.42986, 30.42750 -10.440...",NaN
2,87344b340ffffff,25.3128,-16.2237,8.0000,Southern,3,"POLYGON ((25.32539 -16.22858, 25.31506 -16.238...",NaN
3,87344b340ffffff,25.3128,-16.2237,9.0000,Western,4,"POLYGON ((25.32539 -16.22858, 25.31506 -16.238...",NaN


In [18]:
mines.head(4)

,Country,FeatureNam,DsgAttr03,DsgAttr06,MemoOther,MemoLoc,Latitude,Longitude,DsgAttr07,Output type (ass.),...,Cu content [kt],Mine type,Metal processing,Smelter type,Spec energy Elec [GJ/t],Spec energy Diesel [GJ/t],Energy Elec [TJ],Energy Diesel [TJ],id,geometry
0,Zambia,Baluba Center Underground Mine,Concentrate,2017,Ore. Operations suspended. Facility on care-an...,<null>,-13.0486,28.3381,"1,500,000.0000",Ore and concentrate,...,9.3000,Underground,,,0.0728,0.0133,109.2750,19.9950,1,POINT (28.33806 -13.04861)
1,Zambia,Bwana Mkubwa Solvent Extraction-Electrowinning...,Metal,2017,Copper cathode.,<null>,-13.0423,28.7094,"52,000.0000",Metal,...,26.0000,,Smelter+Refinery,"Solvent Extraction-Electrowinning, Electrowinn...",12.4660,1.5180,324.1160,39.4680,2,POINT (28.70935 -13.04230)
2,Zambia,Chambishi Facility,Metal,2017,Copper cathode.,<null>,-12.6500,28.0412,"8,000.0000",Metal,...,4.0000,,Smelter+Refinery,,12.4660,1.5180,49.8640,6.0720,3,POINT (28.04117 -12.64998)
3,Zambia,Chambishi Cobalt Plant,Metal,2017,Copper cathode. Nkana Slag dumps.,75 Kilometers northwest of Ndola,-12.6500,28.0412,"27,000.0000",Metal,...,13.5000,,Smelter+Refinery,,12.4660,1.5180,168.2910,20.4930,4,POINT (28.04117 -12.64998)


# Part 2. Compute demand

In [19]:
# Enegery balance from UN Stats
energyBalance_path = "EnergyBalance/"
file_energyBalance = "UNSD+DF_UNData_EnergyBalance+1.0_Zambia.csv"
eb = pd.read_csv(energyBalance_path + file_energyBalance)
code_elec = "B07_EL"
code_oil = "B03_OP"
code_ind_nFM=  "B29_1214a"
code_ind_mining =  "B33_1214e"

elec_nonFerrousMetals_TJ = eb.loc[(eb['COMMODITY'] == code_elec) & (eb['TRANSACTION'] == code_ind_nFM) & (eb['TIME_PERIOD'] == 2019 ), 'OBS_VALUE'] #TJ
elec_mining_TJ = eb.loc[(eb['COMMODITY'] == code_elec) & (eb['TRANSACTION'] == code_ind_mining) & (eb['TIME_PERIOD'] == 2019 ), 'OBS_VALUE'] #TJ
oil_nonFerrousMetals_TJ = eb.loc[(eb['COMMODITY'] == code_oil) & (eb['TRANSACTION'] == code_ind_nFM) & (eb['TIME_PERIOD'] == 2019 ), 'OBS_VALUE'] #TJ
oil_mining_TJ = eb.loc[(eb['COMMODITY'] == code_oil) & (eb['TRANSACTION'] == code_ind_mining) & (eb['TIME_PERIOD'] == 2019 ), 'OBS_VALUE'] #TJ

elec_nonFerrousMetals_TJ = pd.to_numeric(elec_nonFerrousMetals_TJ.str.replace(',', '')) # convert to numeric and remove commas
elec_nonFerrousMetals_TJ = elec_nonFerrousMetals_TJ.iloc[0]
elec_mining_TJ = pd.to_numeric(elec_mining_TJ.str.replace(',', '')) # convert to numeric and remove commas
elec_mining_TJ = elec_mining_TJ.iloc[0]
elec = elec_nonFerrousMetals_TJ+elec_mining_TJ
oil_nonFerrousMetals_TJ = pd.to_numeric(oil_nonFerrousMetals_TJ.str.replace(',', '')) # convert to numeric and remove commas
oil_nonFerrousMetals_TJ = oil_nonFerrousMetals_TJ.iloc[0]
oil_mining_TJ = pd.to_numeric(oil_mining_TJ.str.replace(',', '')) # convert to numeric and remove commas
oil_mining_TJ = oil_mining_TJ.iloc[0]
oil = oil_nonFerrousMetals_TJ+oil_mining_TJ

energy = elec+oil

print("total statistical electricity consumption of nonFerrousMetals:",f"{elec_nonFerrousMetals_TJ:,.1f}", "TJ")
print("total statistical electricity consumption in mining:",f"{elec_mining_TJ:,.1f}", "TJ")
print("total statistical electricity consumption of nonFerrousMetals and mining:",f"{elec:,.1f}", "TJ")

print("total statistical oil consumption of nonFerrousMetals:",f"{oil_nonFerrousMetals_TJ:,.1f}", "TJ")
print("total statistical oil consumption in mining:",f"{oil_mining_TJ:,.1f}", "TJ")
print("total statistical oil consumption of nonFerrousMetals and mining:",f"{oil:,.1f}", "TJ")

print("total statistical  energy consumption of nonFerrousMetals and mining:",f"{energy:,.1f}", "TJ")

total statistical nonFerrousMetals electricity consumption: 22,897.0 TJ or  6.4 TWh
total statistical mining electricity consumption: 818.0 TJ or  0.2 TWh
total statistical nonFerrousMetals and mining electricity consumption: 23,715.0 TJ or  6.6 TWh


In [20]:
# Assess total energy consumption and total production
total_elec_consum = mines[columnName_ElecConsumptionMines].sum() # total
total_oil_consum = mines[columnName_OilConsumptionMines].sum() # total
total_energy_consum = total_elec_consum + total_oil_consum
coverage_energy_nonFe_mining = total_energy_consum/energy

# print("total production:", f"{total_production/10**3:,.0f}", "kt")
print("total calculated electricity ", f"{total_elec_consum:,.1f}",", diesel ", f"{total_oil_consum:,.1f}", " and energy consumption:", f"{total_energy_consum:,.1f}", "TJ")
print("Coverage energy in total statistical nonFerrousMetals and mining consumption:", f"{coverage_energy_nonFe_mining*100:,.0f}", "%")

total calculated energy consumption: 17,986.6 TJ or  5.0 TWh
Coverage energy in total statistical nonFerrousMetals and mining electricity consumption: 76 %


In [21]:
#Allocate to each hexagon the industry energy consumption
#grid["IndEnergy"]=grid[columnNameMines]*energycons_perton
# adjust energy consumption if coverage is the whole country
if area == "COUNTRY":
    grid["IndEne_TJ"]=grid[columnName_ElecConsumptionMines]/coverage_elec_nonFe_mining # TJ
else:
    grid["IndEne_TJ"]=grid[columnName_ElecConsumptionMines] # TJ
grid.head(3)

,hexagons,lon,lat,index_righ,NAME_1,id,geometry,Energy Elec [TJ],IndEne_TJ
0,87399485cffffff,30.5240,-12.1759,5.0000,Muchinga,1,"POLYGON ((30.53691 -12.18031, 30.52690 -12.190...",NaN,NaN
1,873995215ffffff,30.4244,-10.4257,7.0000,Northern,2,"POLYGON ((30.43731 -10.42986, 30.42750 -10.440...",NaN,NaN
2,87344b340ffffff,25.3128,-16.2237,8.0000,Southern,3,"POLYGON ((25.32539 -16.22858, 25.31506 -16.238...",NaN,NaN


In [22]:
total_industryenergy = grid["IndEne_TJ"].sum()
#print("Industry electricity consumption:",f"{total_industryenergy/10**6:,.0f}", "TWh")
print("Industry electricity consumption in",f"{area}", ":",f"{total_industryenergy/3600:,.1f}", "TWh")

Industry electricity consumption in COUNTRY : 6.6 TWh


In [23]:
grid.to_file(out_path + 'ind_energy_map.shp', index=False)
grid.to_file(out_path + f'ind_energy_map.geojson', driver='GeoJSON', index=False)  
grid.head(3)

,hexagons,lon,lat,index_righ,NAME_1,id,geometry,Energy Elec [TJ],IndEne_TJ
0,87399485cffffff,30.5240,-12.1759,5.0000,Muchinga,1,"POLYGON ((30.53691 -12.18031, 30.52690 -12.190...",NaN,NaN
1,873995215ffffff,30.4244,-10.4257,7.0000,Northern,2,"POLYGON ((30.43731 -10.42986, 30.42750 -10.440...",NaN,NaN
2,87344b340ffffff,25.3128,-16.2237,8.0000,Southern,3,"POLYGON ((25.32539 -16.22858, 25.31506 -16.238...",NaN,NaN


In [24]:
grid[columnName_ElecConsumptionMines]

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
122983   NaN
122984   NaN
122985   NaN
122986   NaN
122987   NaN
Name: Energy Elec [TJ], Length: 122988, dtype: float64